In [1]:
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
S = PowerSeriesRing(QQ,'x', default_prec=1000)
L=LaurentSeriesRing(QQ,'x', default_prec=1000)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=1000)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.

def uniteList(lst):
    ln=len(lst)
    if ln<2:return lst
    answer=[]
    for j in [0..ln-1]:
        lj=lst[j]
        if (lj in answer)==False:
            answer=answer+[lj]
    return answer

def delete_element(elt,ls):
    new=[]
    for j in ls:
        if j!=elt:new=new+[j]
    return new

def sortAndSmash(lst): #sorts and deletes repeats
    cropped=lst
    new=[]
    while len(cropped)>0:
        m=min(cropped)
        new=new+[m]
        cropped=delete_element(m,cropped)
    return(new)

def difference_table(lst):
    diffs=[]
    for k in [0..len(lst)-2]:
        diff=lst[k+1]-lst[k]
        diffs=diffs+[diff]
    return diffs

def evaluate(expression,incomingvalue):
    expr=str(expression)
    output=sage_eval(expr,locals={'x':incomingvalue})
    return output

P=Primes()

def prime(n): return P.unrank(n-1)

def unite(A,B):
    return set(A).union(B)

def factor_integer(n): #this is like "factor(integer)", but
    #the bases and exponents can be easily extracted.
    F=factor(n)
    return list(F)

def orderInteger(p,n):
    if n!=0:
        lst=n.digits(p)
        count=0
        k=0
        while lst[k]==0:
            count=count+1
            k=k+1
        return count
    if n==0:return(Infinity)

def order(p,fr):
    nn=numerator(QQ(fr))
    dn=denominator(QQ(fr))
    return orderInteger(p,nn)-orderInteger(p,dn)

def digitsum(p,n):return add(n.digits(p))

def digitsum(p,n):
    return sum(n.digits(p))

def rmnjntau(n):
    answer=0
    for k in [1..n-1]:
        answer=answer+(6*k-4*(n-k))*sigma(k,3)*sigma(n-k,5)
    answer=(5*sigma(n,3)+7*sigma(n,5))*n/12-35*answer
    return answer

def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])

def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer

def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))

def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))

def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)

def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))

def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)

def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)

def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # 
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.

def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)

def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]

def xjNew(n,m):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(m**3*2**6)**i
        return rt
    return xjNewStep(n-1,m)

def j(n,m):return L(xjNew(n,m)/x)

def xjLeoNew(n,m,multiplier):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(multiplier)**i
        return rt
    return xjNewStep(n-1,m)

def jLeo(n,m,multiplier):return L(xjLeoNew(n,m,multiplier)/x)

def JpowerConstantTerm(m,power):
    trun=J(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def jpowerConstantTerm(m,power):
    trun=j(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def polynomialCoefficient(n,poly):
    return poly.list()[n]

def leadingCoefficient(polynomial):
    dg=polynomial.degree()
    
    return polynomialCoefficient(dg,polynomial)

def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent

def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)

def firstfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.first())

def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())

def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)

def element(k,listo):
    if k>len(listo):return (print("bad k"))
    return listo[k-1]

def mobiusInverseOfFunction(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        f=f+moebius(dvk)*g(n/dvk)
    return f

def mobiusInverseOfList(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        gndvk=element(n/dvk,g)
        f=f+moebius(dvk)*gndvk
    return f

def drop_first_element(listo):
    lenth=len(listo)
    answer=[]
    for k in [1..lenth-1]:
        answer=answer+[listo[k]]
    return answer

def factorSeriesFromLists(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
    return nswr

def factorSeriesFromListsWithTracking(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
            print((n,k))
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
        print("answer length: ",len(nswr))
    return nswr

def EulerMultiplication(listo):
    # Lemma 2.11 from my 1998 Experimental Mathematics paper.
    # Here the set A in the lemma is {1, 2, ...} = Z^+.
    p_A=[1] # so p_A[0]=1 as on top of p. 266, column 2.
    def ex(n):return listo[n-1] # ex is the exponent on the factor 1-x^n: -f(n)/n.
    def f(n):return -n*ex(n)
    answer=[1]
    def f_A(k):
        summ=0
        dk=divisors(k)
        for d in dk: #Here is where A = Z^+ is used to drop a constraint on k.
            summ=summ+f(d)
        return summ
    for n in [1..len(listo)-1]:
        sm=0
        for k in [1..n]:
            sm=sm+f_A(k)*p_A[n-k]
        p_A=p_A+[sm/n]
    return p_A

def listToPoly(listo):
    lenth=len(listo)
    answer=0
    for k in [1..lenth]:
        cf=listo[k-1]
        answer=answer+cf*x^(k-1)
    return answer
    
#Broadhurst code from #A005148:

#? {nt=100; a=[1]; b=[1]; d=1; e=0; g=0; print(1); for(n=2, nt, c=48*(a[n-1]+g)+128*(d-32*e); 
#e=d; d=c; i=(n-1)\2; g=12*if(n%2==0, a[n/2]^2)+24*sum(j=1, i, 
#a[j]*a[n-j]); h=12*if(n%2==0, b[n/2]^2)+24*sum(j=1, i, b[j]*b[n-j]); f=(c+5*h)/n^2-g; a=concat(a, f); 
#b=concat(b, n*f); print(f))}

#produces this on the PARI online evaluator (I have inserted the commas):
br=[
1,
47,
2488,
138799,
7976456,
467232200,
27736348480,
1662803271215,
100442427373480,
6103747246289272,
372725876150863808,
22852464771010647496,
1405886026610765892544,
86741060172969340021952,
5365190340823180439326208,
332577246704242939511725615,
20655377769544663820919905000,
1285027807539621869480480977880,
80066610886753513409821525593280,
4995543732566526565060187887772024,
312067903389730540416319245145039936,
19516459352109724206910675815791735872,
1221787478073080268912138739833447254528,
76558881238278398609546573647116818306504,
4801399849802188285872546222298724299377856,
301358552889212442951924121355286655092791360,
18928524108186605379268259069278244869735006720,
1189719542605042010945455887482239233732751142080,
74824958481405101799295401923145498080031496317440,
4708731584940969251488540213411242070133095720768000,
296483323638911778793802123013217365155428610625064960,
18677571039055424502042574350078071038555962934810664495,
1177200955467256907707767829606512556434525730284672082280,
74229820742983998523807878655148660941364964757170232076440,
4682657672641000613276353688819373189604961982881761635174080,
295516785862704112676947743865736338547152307208873658542187480,
18656838683258040776726836797753969443154060448210951169536087360,
1178287550937265649491805466460363896744099593833261406542090821440,
74441259433548426510664621182339422182178689134172479673100078686720,
4704546876230537649051669928635037299315044055233418643313504347890040,
297410696380227510473584821926459754598587577997951261584830786025989440,
18807176292551896455842616399574167855948518855982280636468413444438841280,
1189632505858785415664268185396568316810012962868095237190924015678644805120,
75269436592700558660145646818728077669744495747378078929068356710829357904960,
4763606735739477078702262301306618196904330454342036172567804617626114845601280,
301550219357655322958904198748139655940272138707157414253528789096123355242370560,
19093491105382437947961430595496009051927469794600124607374594862297809973497425920,
1209229421833128214532165231904398024088456532579184673374765702204525386892709582280,
76599462222171488217469562807555444840329820375936645628428503967599842536403748392640,
4853249476279584943018752544135518205835823652569328104071808597099976302206777672382272,
307557161322636240130429378891312616501367843861094990590342588813640658716687276702796288,
19494049310445477564550759598016449348244789590449694853790210431694353584090244058674665536,
1235828235382992966112000337356874542591602729215314383988292324949451319922660010288507026944,
78359386541981263753587663605771220395173695760309511724942887619218844604408365062990458445312,
4969331730256147172835336645727281480799885420753251882375600240456687485460922105379475213365248,
315192836223790555211163444188645825759965673786927108224693473418739558121668975477385609064230080,
19995098780164839843271975968833674908892528077762766665489021901583285029365260596621852077438476800,
1268636632413268672729488723673519658349225434504337043331170274695193536624267473803997222986457986560,
80503572319005599896519802085748511261953758070993177770252662999751870126880338970620619030455487016960,
5109226023515231795174271806642022238278119546486922652746346463140324125835625762209193286894097317188096,
324306079896787161344357124063601140357997183120543361083780244695916059297903737976316852087325736124084224,
20587951238469144728875413572545165973124146169862711202444824611729329948630917798996527215521428049827442688,
1307155886598940172607280751052393343723376964347599361863473016335357927316478471270092962849033836059872428032,
83003437890819298061159220674050513337514687509034267017635549288525596646068731914307086031798152640655708675631,
5271297646867701243231620056348828996858362139183014747261830107569871776680492045283621280463257824831450361228904,
334803625831811164273307471398151444570380979015726639675077730502474540099263374815508109958714263034979870045590040,
21267301347069209180434546119316800827596753601228462222166318226579132663526299067285136882052633679235269979706997440,
1351085301766957831759285167329838127692235945761375100182242807152550787271816136969058195839695684514251809964546971800,
85842015014782910888737358332857326909867676530325317899274801087388530640486646412863052257542933472850576480889534912320,
5454594181103314395353884318297045062026694474478123129221277721208935931363965378785241897523306877686001373372245607417152,
346632363267943630179319892957146981236491202095299908808662717047804243504180203803669349935923567992160546743779715198734848,
22030211252839896655760044436342222596667157343136792244243996326108725373610974640845764035532176284454550965374859141605876696,
1400264005045227498693320089227035884825680282492704474103652886750979292873550448366299419926933438094105780187281431855082851904,
89010607580108363146057947796447019984406819117601908324673767705214888723143847657440514237648023072092581303730690468541358446272,
5658653431087770147783431973072442739707731119479989465299403213540757386414610090412270459203277370032300491403324236240673058231808,
359768282283136451938233183643880156808956764689158714079901896700007719423392215459057530585195812985027222576505521483746145927094080,
22875473496936829687402292014161718834412927763836095316924893494178776078421448526507427925257601899589958056800447532066174066611688960,
1454634188002381701560543002374509285000151863499255133979293536524595484779236558931041033869616374153934389337102738325693541500885040640,
92506668310411752697700027500541107599054510647853447997767226456825933551220718898889759216482942932408592031625495444638507054727309701120,
5883380647402071727818124358464357357061997088366913641247755161122708769301392216445412497947130792332367814389247083022141893681918673035640,
374209367072642703061850255891790471856520862005092473938011251040764312378697398309875157737634899606364054132972840806767260748645213681887040,
23803199270417251468737963122626707467816366171461192749563400557478169477201165362512851772159448598935076348960799452562007578125691327998310080,
1514217230105491006416286735192399705713778112569360921940868647785308212475854144353099514116567802127049424293324092083010142613089652094239352320,
96332413045374746200370571514086365134691986056054938357641223739177015618938978964850794817403946266783867698429445521201166592631737075434674509760,
6128968041798698464871983060993177701445048464359248049771600533082817745158394358756109989641756654653420473533147391458535031474858708357818557350400,
389970917973786025826826148232023527188983145106526374860563378061488842725970203994300386046448714269543873579140545202535929354547230426525139471434240,
24814546341676335548842726357075976008312393230217108797033854347814848781823641548406242672165084256308341602099263161325277248861481206883551599766753280,
1579097865621621078127966349210038916312383695762720347829578839794195077696910123713050602263102661325782769915208166009373762048848807388370046579798291520,
100493898862353192639599085772479279507605308104393463045761183781449039477596456154787934584841158653853713852389459365917827655606822211804400142188630504960,
6395841086583513721518708585123859851483827511052356442692606484016955654747071476807355907729334535301712240745743084597082858081715137457733426771133436244480,
407082422122816769108928976084224767774083074925265772652509091404070682745260601092117551854989868576150378265801324011663291789547097044490460289150388722831360,
25911536639811395996751656925990326528992266850810482361355730044302538357241007798389242049209348961586470395969302758332164645822911593670231053397678686836037120,
1649413547559894591700394303117536943729317363777978884992676745553087694293098678096611231647446457954863017134436134803435906819926459313995976199288693719229255680,
105000403824773770333649105789820480034335517728840968988334310643726824123735041790377778857595055184234699203779438530862092851011368656739154398110404364696677847040,
6684622341158487811999668275612452018461145981041862416511208589860402818472403829783593014653257049064920715626593324371966426230798803277918448269747031588460412436480,
425585443813049916737778209891287760908166309712249413374261870396364033955750883326966977701620958916811619741413201896787531939944010422188516605008297556440177053390280,
27096933232548903092613115617140470313230857234611666991627504070866204307527763605986822544358886296299818124185350044201435781986714935884426869011696899042312957811757760,
1725347275200146778788065707502165945462707429554800950293313680035690477290460889039398857128345130378453865634762702538349329424603778897699271642610584658725955102258591040,
109864008941653383497254010859226498595190791378492674731068039950918263737791469856053624392850573136737164626778125500188292591996295723267255817207828065983109119673561029120,
6996107097499132971946127571962655485323611430800974100070703369809020527942862074490096967477739069312314066266444140870911945979977377211097174725520015644037734486845682825024]

def broadhurst(n):return br[n-1]

def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})

print("okay1")

okay1


In [8]:
for n in [1..5]:
    print((n,sigma(n,k=11)))

(1, 1)
(2, 2049)
(3, 177148)
(4, 4196353)
(5, 48828126)


In [2]:
import random
import time


def specialModulus(a):# I alter this definition, relative to the previous cell.
    ds=digitsum(2,a)
    return 2*8^ds

def p2RamanujanCongruenceSeries(n): # Ramanujan Revisted, p. 293
    ans=0
    mlist=[]
    for a in [1..n+1]:
        mlist=mlist+[specialModulus(a)]
    sm=max(mlist) # probably bc max(mlist) = lcm(mlist) 
    for a in [1..n+1]:
        r=ZZ(random.randint(-50, 50))
        o2=order(2,a)
        purepart=2^o2
        oddpart=a/purepart
        so=sigma(oddpart,k=11)
        #ord(2,tau(2^n))= (I believe) 2n
        # so ord(2,tau(purepart))=ord(2,tau(2^o2)) = 2*o2
        # so tau(purepart) = 2^(2*o2)
        # so tau(a) = tau(purepart)*tau(oddpart)=
        # 4^o2*tau(oddpart); then we replace tau(oddpart)
        # with sigma(oddpart,k=11) by Ramanujan's congruence
        if mod(oddpart,8)==1:d=so
        if mod(oddpart,8)==3:d=1217*so
        if mod(oddpart,8)==5:d=1537*so
        if mod(oddpart,8)==7:d=705*so
        term=d*4^o2++r*sm        
        ans=ans+term*x^a
    return D(ans)

def recipP2(n):
    ans=L(1/p2RamanujanCongruenceSeries(n))
    ans=ans.O(n+1)
    return ans

def p2RecipPowerConstantTerm(k):
    g=recipP2(k)^k
    g=g.O(k+1)
    #print("g:");print(g);print()
    #print("list:")
    gl=g.list()
    #print(gl);print()
    ln=len(gl)
    return gl[ln-2]
    

L=LaurentSeriesRing(QQ,'x', default_prec=1010)

no=[]
for b in [1..5]:
    print()
    ls=[]
    for a in [1..1000]: # 1000 < precision is necessary
        start=time.time()
        c=p2RecipPowerConstantTerm(a)
        o2=order(2,c)
        ds=digitsum(2,a)
        test=o2-3*ds+order(2,a+1) # 3rd term based on some observations
        if test!=0:
            ls=ls+[a]
            no=no+[(b,a)]
        finish=time.time()
        print((b,a,test,len(no),finish-start))
    print()
    print("bad a's for trial ",b);print();print(ls)
print("------------------------------------------------------------------")
print("no:");print();print(no)


(1, 1, 0, 0, 0.08091115951538086)
(1, 2, 0, 0, 0.13646697998046875)
(1, 3, 0, 0, 0.13863921165466309)
(1, 4, 0, 0, 0.08431577682495117)
(1, 5, 0, 0, 0.11342597007751465)
(1, 6, 0, 0, 0.18144989013671875)
(1, 7, 0, 0, 0.2140827178955078)
(1, 8, 0, 0, 0.19764924049377441)
(1, 9, 0, 0, 0.2082979679107666)
(1, 10, 0, 0, 0.21059608459472656)
(1, 11, 0, 0, 0.1989150047302246)
(1, 12, 0, 0, 0.13706493377685547)
(1, 13, 0, 0, 0.21098089218139648)
(1, 14, 0, 0, 0.2556459903717041)
(1, 15, 0, 0, 0.19436120986938477)
(1, 16, 0, 0, 0.24416708946228027)
(1, 17, 0, 0, 0.24467897415161133)
(1, 18, 0, 0, 0.19718098640441895)
(1, 19, 0, 0, 0.24107813835144043)
(1, 20, 0, 0, 0.23369193077087402)
(1, 21, 0, 0, 0.23010015487670898)
(1, 22, 0, 0, 0.23703908920288086)
(1, 23, 0, 0, 0.24170899391174316)
(1, 24, 0, 0, 0.18552017211914062)
(1, 25, 0, 0, 0.2621147632598877)
(1, 26, 0, 0, 0.23781871795654297)
(1, 27, 0, 0, 0.26787686347961426)
(1, 28, 0, 0, 0.2089390754699707)
(1, 29, 0, 0, 0.23548412322998047)

(1, 236, 0, 0, 0.5935709476470947)
(1, 237, 0, 0, 0.6039190292358398)
(1, 238, 0, 0, 0.6125500202178955)
(1, 239, 0, 0, 0.5188679695129395)
(1, 240, 0, 0, 0.5285120010375977)
(1, 241, 0, 0, 0.5684669017791748)
(1, 242, 0, 0, 0.5660068988800049)
(1, 243, 0, 0, 0.636486291885376)
(1, 244, 0, 0, 0.5609512329101562)
(1, 245, 0, 0, 0.615710973739624)
(1, 246, 0, 0, 0.6247379779815674)
(1, 247, 0, 0, 0.5776450634002686)
(1, 248, 0, 0, 0.5800521373748779)
(1, 249, 0, 0, 0.6313400268554688)
(1, 250, 0, 0, 0.6065347194671631)
(1, 251, 0, 0, 0.6492478847503662)
(1, 252, 0, 0, 0.6308541297912598)
(1, 253, 0, 0, 0.6501297950744629)
(1, 254, 0, 0, 0.7229492664337158)
(1, 255, 0, 0, 0.7091829776763916)
(1, 256, 0, 0, 0.7022039890289307)
(1, 257, 0, 0, 0.5970089435577393)
(1, 258, 0, 0, 0.7037019729614258)
(1, 259, 0, 0, 0.6019210815429688)
(1, 260, 0, 0, 0.5852620601654053)
(1, 261, 0, 0, 0.6965501308441162)
(1, 262, 0, 0, 0.6745619773864746)
(1, 263, 0, 0, 0.746279239654541)
(1, 264, 0, 0, 0.725671

(1, 472, 0, 0, 1.4939870834350586)
(1, 473, 0, 0, 1.206866979598999)
(1, 474, 0, 0, 1.550903081893921)
(1, 475, 0, 0, 1.5102789402008057)
(1, 476, 0, 0, 1.3217997550964355)
(1, 477, 0, 0, 1.571925163269043)
(1, 478, 0, 0, 1.6759040355682373)
(1, 479, 0, 0, 1.666395902633667)
(1, 480, 0, 0, 1.4641611576080322)
(1, 481, 0, 0, 1.4574708938598633)
(1, 482, 0, 0, 1.472809076309204)
(1, 483, 0, 0, 1.4666099548339844)
(1, 484, 0, 0, 1.4289579391479492)
(1, 485, 0, 0, 1.6042349338531494)
(1, 486, 0, 0, 1.410904884338379)
(1, 487, 0, 0, 1.3874187469482422)
(1, 488, 0, 0, 1.3861157894134521)
(1, 489, 0, 0, 1.6067657470703125)
(1, 490, 0, 0, 1.4562551975250244)
(1, 491, 0, 0, 1.6906981468200684)
(1, 492, 0, 0, 1.3650779724121094)
(1, 493, 0, 0, 1.70051908493042)
(1, 494, 0, 0, 1.589416742324829)
(1, 495, 0, 0, 1.761117935180664)
(1, 496, 0, 0, 1.3640587329864502)
(1, 497, 0, 0, 0.6861329078674316)
(1, 498, 0, 0, 1.5261039733886719)
(1, 499, 0, 0, 1.6919622421264648)
(1, 500, 0, 0, 1.6527340412139

(1, 710, 0, 0, 3.445694923400879)
(1, 711, 0, 0, 3.3492531776428223)
(1, 712, 0, 0, 3.106215000152588)
(1, 713, 0, 0, 3.3100271224975586)
(1, 714, 0, 0, 3.5028090476989746)
(1, 715, 0, 0, 3.7381317615509033)
(1, 716, 0, 0, 3.303838014602661)
(1, 717, 0, 0, 3.3146958351135254)
(1, 718, 0, 0, 3.494138240814209)
(1, 719, 0, 0, 3.7324161529541016)
(1, 720, 0, 0, 3.279493808746338)
(1, 721, 0, 0, 3.2488341331481934)
(1, 722, 0, 0, 3.5475540161132812)
(1, 723, 0, 0, 3.8240561485290527)
(1, 724, 0, 0, 3.580897092819214)
(1, 725, 0, 0, 3.8075010776519775)
(1, 726, 0, 0, 3.355926036834717)
(1, 727, 0, 0, 4.029707193374634)
(1, 728, 0, 0, 3.512779951095581)
(1, 729, 0, 0, 3.7487070560455322)
(1, 730, 0, 0, 3.6353793144226074)
(1, 731, 0, 0, 1.7453019618988037)
(1, 732, 0, 0, 1.6542181968688965)
(1, 733, 0, 0, 3.617633104324341)
(1, 734, 0, 0, 4.081137180328369)
(1, 735, 0, 0, 4.355492830276489)
(1, 736, 0, 0, 3.5004518032073975)
(1, 737, 0, 0, 3.2769317626953125)
(1, 738, 0, 0, 3.765068054199218

(1, 951, 0, 0, 6.959925174713135)
(1, 952, 0, 0, 6.5644872188568115)
(1, 953, 0, 0, 6.871086835861206)
(1, 954, 0, 0, 7.04131293296814)
(1, 955, 0, 0, 6.810906171798706)
(1, 956, 0, 0, 6.450102090835571)
(1, 957, 0, 0, 6.7375710010528564)
(1, 958, 0, 0, 6.8602190017700195)
(1, 959, 0, 0, 7.912770986557007)
(1, 960, 0, 0, 5.7710120677948)
(1, 961, 0, 0, 5.73121190071106)
(1, 962, 0, 0, 6.242408752441406)
(1, 963, 0, 0, 6.116558790206909)
(1, 964, 0, 0, 6.206885814666748)
(1, 965, 0, 0, 6.158125162124634)
(1, 966, 0, 0, 6.694954872131348)
(1, 967, 0, 0, 6.545475959777832)
(1, 968, 0, 0, 6.20636510848999)
(1, 969, 0, 0, 6.2775208950042725)
(1, 970, 0, 0, 6.171365976333618)
(1, 971, 0, 0, 7.161549806594849)
(1, 972, 0, 0, 6.179861783981323)
(1, 973, 0, 0, 6.443583011627197)
(1, 974, 0, 0, 7.186537981033325)
(1, 975, 0, 0, 7.365555047988892)
(1, 976, 0, 0, 5.8470470905303955)
(1, 977, 0, 0, 6.696200370788574)
(1, 978, 0, 0, 6.774407148361206)
(1, 979, 0, 0, 7.312398672103882)
(1, 980, 0, 0,

...

(5, 38, 0, 0, 0.30510926246643066)
(5, 39, 0, 0, 0.28590822219848633)
(5, 40, 0, 0, 0.3200550079345703)
(5, 41, 0, 0, 0.26900410652160645)
(5, 42, 0, 0, 0.22711610794067383)
(5, 43, 0, 0, 0.28720998764038086)
(5, 44, 0, 0, 0.2768561840057373)
(5, 45, 0, 0, 0.2858438491821289)
(5, 46, 0, 0, 0.28824806213378906)
(5, 47, 0, 0, 0.28594493865966797)
(5, 48, 0, 0, 0.2942368984222412)
(5, 49, 0, 0, 0.22867584228515625)
(5, 50, 0, 0, 0.2819540500640869)
(5, 51, 0, 0, 0.23720574378967285)
(5, 52, 0, 0, 0.31014299392700195)
(5, 53, 0, 0, 0.2820279598236084)
(5, 54, 0, 0, 0.2882862091064453)
(5, 55, 0, 0, 0.29261279106140137)
(5, 56, 0, 0, 0.22384381294250488)
(5, 57, 0, 0, 0.267578125)
(5, 58, 0, 0, 0.2782890796661377)
(5, 59, 0, 0, 0.2759418487548828)
(5, 60, 0, 0, 0.3029441833496094)
(5, 61, 0, 0, 0.2908899784088135)
(5, 62, 0, 0, 0.3451666831970215)
(5, 63, 0, 0, 0.38560986518859863)
(5, 64, 0, 0, 0.2791569232940674)
(5, 65, 0, 0, 0.3558030128479004)
(5, 66, 0, 0, 0.3507840633392334)
(5, 67, 

(5, 273, 0, 0, 0.6444637775421143)
(5, 274, 0, 0, 0.7684500217437744)
(5, 275, 0, 0, 0.5804071426391602)
(5, 276, 0, 0, 0.7411172389984131)
(5, 277, 0, 0, 0.7883687019348145)
(5, 278, 0, 0, 0.8020899295806885)
(5, 279, 0, 0, 0.7420449256896973)
(5, 280, 0, 0, 0.714216947555542)
(5, 281, 0, 0, 0.7484819889068604)
(5, 282, 0, 0, 0.7967431545257568)
(5, 283, 0, 0, 0.7630720138549805)
(5, 284, 0, 0, 0.7754387855529785)
(5, 285, 0, 0, 0.7383317947387695)
(5, 286, 0, 0, 0.7690300941467285)
(5, 287, 0, 0, 0.8106789588928223)
(5, 288, 0, 0, 0.6875350475311279)
(5, 289, 0, 0, 0.7484347820281982)
(5, 290, 0, 0, 0.7155101299285889)
(5, 291, 0, 0, 0.7079029083251953)
(5, 292, 0, 0, 0.7612190246582031)
(5, 293, 0, 0, 0.7256641387939453)
(5, 294, 0, 0, 0.7731521129608154)
(5, 295, 0, 0, 0.7586090564727783)
(5, 296, 0, 0, 0.738347053527832)
(5, 297, 0, 0, 0.7829470634460449)
(5, 298, 0, 0, 0.8277032375335693)
(5, 299, 0, 0, 0.8688828945159912)
(5, 300, 0, 0, 0.8112320899963379)
(5, 301, 0, 0, 0.80908

(5, 509, 0, 0, 1.8824610710144043)
(5, 510, 0, 0, 2.026212215423584)
(5, 511, 0, 0, 2.250420093536377)
(5, 512, 0, 0, 1.5523929595947266)
(5, 513, 0, 0, 1.644657850265503)
(5, 514, 0, 0, 1.6749367713928223)
(5, 515, 0, 0, 1.7648980617523193)
(5, 516, 0, 0, 1.626532793045044)
(5, 517, 0, 0, 1.7778596878051758)
(5, 518, 0, 0, 1.7606620788574219)
(5, 519, 0, 0, 1.7030551433563232)
(5, 520, 0, 0, 1.659717082977295)
(5, 521, 0, 0, 1.8449862003326416)
(5, 522, 0, 0, 1.8359529972076416)
(5, 523, 0, 0, 1.8758409023284912)
(5, 524, 0, 0, 1.8639748096466064)
(5, 525, 0, 0, 1.6587369441986084)
(5, 526, 0, 0, 1.869826078414917)
(5, 527, 0, 0, 2.022449254989624)
(5, 528, 0, 0, 1.7156479358673096)
(5, 529, 0, 0, 1.8164942264556885)
(5, 530, 0, 0, 1.8420321941375732)
(5, 531, 0, 0, 1.819978952407837)
(5, 532, 0, 0, 1.8696999549865723)
(5, 533, 0, 0, 1.8796169757843018)
(5, 534, 0, 0, 1.886782169342041)
(5, 535, 0, 0, 2.0053870677948)
(5, 536, 0, 0, 1.736285924911499)
(5, 537, 0, 0, 1.8303608894348145

(5, 747, 0, 0, 3.9195621013641357)
(5, 748, 0, 0, 3.83449387550354)
(5, 749, 0, 0, 4.1701979637146)
(5, 750, 0, 0, 3.639172077178955)
(5, 751, 0, 0, 4.40012001991272)
(5, 752, 0, 0, 3.789118766784668)
(5, 753, 0, 0, 4.060388088226318)
(5, 754, 0, 0, 4.268169164657593)
(5, 755, 0, 0, 4.302170038223267)
(5, 756, 0, 0, 3.9851338863372803)
(5, 757, 0, 0, 4.302199840545654)
(5, 758, 0, 0, 3.9344089031219482)
(5, 759, 0, 0, 4.732517242431641)
(5, 760, 0, 0, 3.964776039123535)
(5, 761, 0, 0, 4.126552104949951)
(5, 762, 0, 0, 3.681015729904175)
(5, 763, 0, 0, 4.534948825836182)
(5, 764, 0, 0, 4.544692277908325)
(5, 765, 0, 0, 4.525288105010986)
(5, 766, 0, 0, 4.3656229972839355)
(5, 767, 0, 0, 4.775010824203491)
(5, 768, 0, 0, 3.3596811294555664)
(5, 769, 0, 0, 3.0939221382141113)
(5, 770, 0, 0, 3.3439018726348877)
(5, 771, 0, 0, 3.669663906097412)
(5, 772, 0, 0, 3.5779969692230225)
(5, 773, 0, 0, 3.6229541301727295)
(5, 774, 0, 0, 3.692863941192627)
(5, 775, 0, 0, 4.13490891456604)
(5, 776, 0

(5, 988, 0, 0, 7.002581834793091)
(5, 989, 0, 0, 6.9990153312683105)
(5, 990, 0, 0, 6.992906332015991)
(5, 991, 0, 0, 8.301481008529663)
(5, 992, 0, 0, 6.4783360958099365)
(5, 993, 0, 0, 6.945019006729126)
(5, 994, 0, 0, 6.8974809646606445)
(5, 995, 0, 0, 7.624440908432007)
(5, 996, 0, 0, 6.712784051895142)
(5, 997, 0, 0, 7.418000936508179)
(5, 998, 0, 0, 7.678270101547241)
(5, 999, 0, 0, 7.8112781047821045)
(5, 1000, 0, 0, 7.220006704330444)

bad a's for trial  5

[]
------------------------------------------------------------------
no:

[]


In [6]:
# verify narrow version
import time


def p2RamanujanCongruenceSeries(n): # Ramanujan Revisted, p. 293
    ans=0
    for a in [1..n+1]:
        o2=order(2,a)
        purepart=2^o2
        oddpart=a/purepart
        #ord(2,tau(2^n))= (I believe) 2d
        # so ord(2,tau(purepart))=ord(2,tau(2^o2)) = 2*o2
        # so tau(purepart) = 2^(2*o2)
        # so tau(a) = tau(purepart)*tau(oddpart)=
        # 4^o2*tau(oddpart); then we replace tau(oddpart)
        # with sigma(oddpart,k=11) by Ramanujan's congruence
        if mod(oddpart,8)==1:d=sigma(oddpart,k=11)
        if mod(oddpart,8)==3:d=1217*sigma(oddpart,k=11)
        if mod(oddpart,8)==5:d=1537*sigma(oddpart,k=11)
        if mod(oddpart,8)==7:d=705*sigma(oddpart,k=11)
        term=d*4^o2*x^a        
        ans=ans+term
    return D(ans)

def recipP2(n):
    ans=L(1/p2RamanujanCongruenceSeries(n))
    ans=ans.O(n+1)
    return ans

def p2RecipPowerConstantTerm(k):
    g=recipP2(k)^k
    g=g.O(k+1)
    #print("g:");print(g);print()
    #print("list:")
    gl=g.list()
    #print(gl);print()
    ln=len(gl)
    return gl[ln-2]
    

L=LaurentSeriesRing(QQ,'x', default_prec=2010)

ls=[]
start=time.time()
for a in [1..2000]: #default precision must be > max a.
    c=p2RecipPowerConstantTerm(a)
    o2=order(2,c)
    ds=digitsum(2,a)
    test=o2-3*ds+order(2,a+1) # 3rd term based on some observations
    # this will verify the 'rules' for even a.
    if test!=0:ls=ls+[a]
    finish=time.time()
    print((a,test,len(ls),(finish-start)/60))
print()
print("ls:");print();print(ls)
print("len(ls):",len(ls))

(1, 0, 0, 0.00012618700663248698)
(2, 0, 0, 0.0009840170542399088)
(3, 0, 0, 0.00199282169342041)
(4, 0, 0, 0.0031497001647949217)
(5, 0, 0, 0.004498116175333659)
(6, 0, 0, 0.006485386689503988)
(7, 0, 0, 0.008187603950500489)
(8, 0, 0, 0.009891621271769206)
(9, 0, 0, 0.011806984742482504)
(10, 0, 0, 0.013502136866251627)
(11, 0, 0, 0.015179018179575602)
(12, 0, 0, 0.016873188813527427)
(13, 0, 0, 0.018787288665771486)
(14, 0, 0, 0.020512620608011883)
(15, 0, 0, 0.022461469968159994)
(16, 0, 0, 0.024176935354868572)
(17, 0, 0, 0.026098954677581786)
(18, 0, 0, 0.02788370450337728)
(19, 0, 0, 0.029577553272247314)
(20, 0, 0, 0.03155981699625651)
(21, 0, 0, 0.03339770237604777)
(22, 0, 0, 0.0354984720547994)
(23, 0, 0, 0.03740113576253255)
(24, 0, 0, 0.03928020397822062)
(25, 0, 0, 0.041352883974711103)
(26, 0, 0, 0.043209536870320635)
(27, 0, 0, 0.04506462017695109)
(28, 0, 0, 0.04724930127461751)
(29, 0, 0, 0.0491147518157959)
(30, 0, 0, 0.05096946954727173)
(31, 0, 0, 0.052828450997670

(256, 0, 0, 0.6238399386405945)
(257, 0, 0, 0.6271292686462402)
(258, 0, 0, 0.6302165706952413)
(259, 0, 0, 0.6336100379625956)
(260, 0, 0, 0.6368076841036479)
(261, 0, 0, 0.640107540289561)
(262, 0, 0, 0.6435183207194011)
(263, 0, 0, 0.6466798861821492)
(264, 0, 0, 0.6499486049016316)
(265, 0, 0, 0.6532979369163513)
(266, 0, 0, 0.6566470384597778)
(267, 0, 0, 0.660018789768219)
(268, 0, 0, 0.6633570551872253)
(269, 0, 0, 0.6667741020520528)
(270, 0, 0, 0.6702011704444886)
(271, 0, 0, 0.6737947185834249)
(272, 0, 0, 0.6770351529121399)
(273, 0, 0, 0.6804986556371053)
(274, 0, 0, 0.683793556690216)
(275, 0, 0, 0.6873658061027527)
(276, 0, 0, 0.6907834053039551)
(277, 0, 0, 0.6943497697512309)
(278, 0, 0, 0.6980269193649292)
(279, 0, 0, 0.7016500194867452)
(280, 0, 0, 0.7052576541900635)
(281, 0, 0, 0.7089548865954082)
(282, 0, 0, 0.7126715381940206)
(283, 0, 0, 0.7163280367851257)
(284, 0, 0, 0.720058822631836)
(285, 0, 0, 0.7238668878873189)
(286, 0, 0, 0.7276360551516216)
(287, 0, 0, 

(514, 0, 0, 2.0087428371111553)
(515, 0, 0, 2.0160366694132485)
(516, 0, 0, 2.022772554556529)
(517, 0, 0, 2.030180239677429)
(518, 0, 0, 2.0374297380447386)
(519, 0, 0, 2.0451639691988626)
(520, 0, 0, 2.051976438363393)
(521, 0, 0, 2.059431787331899)
(522, 0, 0, 2.066704289118449)
(523, 0, 0, 2.0746084888776144)
(524, 0, 0, 2.081913085778554)
(525, 0, 0, 2.089627234141032)
(526, 0, 0, 2.0973100344340008)
(527, 0, 0, 2.1055031696955364)
(528, 0, 0, 2.11259450117747)
(529, 0, 0, 2.1203366200129192)
(530, 0, 0, 2.1279175877571106)
(531, 0, 0, 2.135866606235504)
(532, 0, 0, 2.1434324026107787)
(533, 0, 0, 2.1513131539026897)
(534, 0, 0, 2.159196134408315)
(535, 0, 0, 2.1675309538841248)
(536, 0, 0, 2.175418937206268)
(537, 0, 0, 2.1833694696426393)
(538, 0, 0, 2.191474135716756)
(539, 0, 0, 2.199723120530446)
(540, 0, 0, 2.2076451500256855)
(541, 0, 0, 2.2160367727279664)
(542, 0, 0, 2.22436603307724)
(543, 0, 0, 2.233067286014557)
(544, 0, 0, 2.2399740338325502)
(545, 0, 0, 2.24773896932

(775, 0, 0, 4.971302155653635)
(776, 0, 0, 4.984280820687612)
(777, 0, 0, 4.998206321398417)
(778, 0, 0, 5.012467618783315)
(779, 0, 0, 5.02723521788915)
(780, 0, 0, 5.04128585656484)
(781, 0, 0, 5.056066219011942)
(782, 0, 0, 5.07099453608195)
(783, 0, 0, 5.087010502815247)
(784, 0, 0, 5.100225003560384)
(785, 0, 0, 5.114448753992717)
(786, 0, 0, 5.128611354033152)
(787, 0, 0, 5.1439082225163775)
(788, 0, 0, 5.158179704348246)
(789, 0, 0, 5.173770884672801)
(790, 0, 0, 5.188980368773143)
(791, 0, 0, 5.205215668678283)
(792, 0, 0, 5.219567902882894)
(793, 0, 0, 5.235435005029043)
(794, 0, 0, 5.251273755232493)
(795, 0, 0, 5.267902652422587)
(796, 0, 0, 5.2832405050595606)
(797, 0, 0, 5.299390451113383)
(798, 0, 0, 5.315582636992136)
(799, 0, 0, 5.332850337028503)
(800, 0, 0, 5.346407453219096)
(801, 0, 0, 5.360939105351766)
(802, 0, 0, 5.376253072420756)
(803, 0, 0, 5.391940553983052)
(804, 0, 0, 5.40657020409902)
(805, 0, 0, 5.422171620527903)
(806, 0, 0, 5.4377804358800255)
(807, 0, 

(1039, 0, 0, 10.490217053890229)
(1040, 0, 0, 10.512887485822041)
(1041, 0, 0, 10.537536903222401)
(1042, 0, 0, 10.562183101971945)
(1043, 0, 0, 10.589634501934052)
(1044, 0, 0, 10.614480185508729)
(1045, 0, 0, 10.641300837198893)
(1046, 0, 0, 10.667976288000743)
(1047, 0, 0, 10.696536489327748)
(1048, 0, 0, 10.721299469470978)
(1049, 0, 0, 10.748017100493113)
(1050, 0, 0, 10.77479980389277)
(1051, 0, 0, 10.80382953484853)
(1052, 0, 0, 10.830778352419536)
(1053, 0, 0, 10.859527238210042)
(1054, 0, 0, 10.88832883834839)
(1055, 0, 0, 10.919474116961162)
(1056, 0, 0, 10.943078052997588)
(1057, 0, 0, 10.968527770042419)
(1058, 0, 0, 10.994191273053486)
(1059, 0, 0, 11.0215207695961)
(1060, 0, 0, 11.047119621435801)
(1061, 0, 0, 11.074783317248027)
(1062, 0, 0, 11.102552787462871)
(1063, 0, 0, 11.132348485787709)
(1064, 0, 0, 11.15804208914439)
(1065, 0, 0, 11.186051853497823)
(1066, 0, 0, 11.213654383023579)
(1067, 0, 0, 11.243664034207661)
(1068, 0, 0, 11.27174370288849)
(1069, 0, 0, 11.3

(1290, 0, 0, 19.112395270665488)
(1291, 0, 0, 19.15102316935857)
(1292, 0, 0, 19.187631352742514)
(1293, 0, 0, 19.226777939001717)
(1294, 0, 0, 19.26532805363337)
(1295, 0, 0, 19.30736178557078)
(1296, 0, 0, 19.341474850972492)
(1297, 0, 0, 19.378335936864218)
(1298, 0, 0, 19.41496258576711)
(1299, 0, 0, 19.454576734701792)
(1300, 0, 0, 19.49107503493627)
(1301, 0, 0, 19.530110236008962)
(1302, 0, 0, 19.569265234470368)
(1303, 0, 0, 19.61150655349096)
(1304, 0, 0, 19.64833870331446)
(1305, 0, 0, 19.687486986319225)
(1306, 0, 0, 19.726640804608664)
(1307, 0, 0, 19.76849410136541)
(1308, 0, 0, 19.807750101884206)
(1309, 0, 0, 19.85017835299174)
(1310, 0, 0, 19.892202552159628)
(1311, 0, 0, 19.938060502211254)
(1312, 0, 0, 19.972518869241078)
(1313, 0, 0, 20.010056551297506)
(1314, 0, 0, 20.04716848929723)
(1315, 0, 0, 20.087556250890096)
(1316, 0, 0, 20.12491968472799)
(1317, 0, 0, 20.16510945558548)
(1318, 0, 0, 20.206674901644387)
(1319, 0, 0, 20.24970475435257)
(1320, 0, 0, 20.2873653

(1542, 0, 0, 31.66870090564092)
(1543, 0, 0, 31.722729603449505)
(1544, 0, 0, 31.769041005770365)
(1545, 0, 0, 31.819566722710928)
(1546, 0, 0, 31.87000263929367)
(1547, 0, 0, 31.924812205632527)
(1548, 0, 0, 31.975735704104107)
(1549, 0, 0, 32.03068828582764)
(1550, 0, 0, 32.08546036879222)
(1551, 0, 0, 32.14467210372289)
(1552, 0, 0, 32.19175092379252)
(1553, 0, 0, 32.24431462287903)
(1554, 0, 0, 32.29506877263387)
(1555, 0, 0, 32.350046916802725)
(1556, 0, 0, 32.40086758534114)
(1557, 0, 0, 32.455671989917754)
(1558, 0, 0, 32.51056085427602)
(1559, 0, 0, 32.56964143514633)
(1560, 0, 0, 32.62044038375219)
(1561, 0, 0, 32.67549606959025)
(1562, 0, 0, 32.73023145198822)
(1563, 0, 0, 32.78926386833191)
(1564, 0, 0, 32.844376202424364)
(1565, 0, 0, 32.90367517073949)
(1566, 0, 0, 32.96312102079391)
(1567, 0, 0, 33.02744943698247)
(1568, 0, 0, 33.075413386027016)
(1569, 0, 0, 33.127435119946796)
(1570, 0, 0, 33.179979503154755)
(1571, 0, 0, 33.23601312239965)
(1572, 0, 0, 33.2882199009259

(1797, 0, 0, 48.87278093894323)
(1798, 0, 0, 48.94696088631948)
(1799, 0, 0, 49.02719138860702)
(1800, 0, 0, 49.096389969189964)
(1801, 0, 0, 49.17202719052633)
(1802, 0, 0, 49.24710261821747)
(1803, 0, 0, 49.327254235744476)
(1804, 0, 0, 49.40217547019323)
(1805, 0, 0, 49.48210742076238)
(1806, 0, 0, 49.5613937218984)
(1807, 0, 0, 49.64679093360901)
(1808, 0, 0, 49.71579330364863)
(1809, 0, 0, 49.790243887901305)
(1810, 0, 0, 49.86497487227122)
(1811, 0, 0, 49.94521555503209)
(1812, 0, 0, 50.02029438416163)
(1813, 0, 0, 50.10040332078934)
(1814, 0, 0, 50.18124143679937)
(1815, 0, 0, 50.267774085203804)
(1816, 0, 0, 50.34259700377782)
(1817, 0, 0, 50.42325780391693)
(1818, 0, 0, 50.50393545627594)
(1819, 0, 0, 50.58975670337677)
(1820, 0, 0, 50.670168018341066)
(1821, 0, 0, 50.7556404709816)
(1822, 0, 0, 50.841574740409854)
(1823, 0, 0, 50.9336482723554)
(1824, 0, 0, 51.00418930451075)
(1825, 0, 0, 51.07997980515162)
(1826, 0, 0, 51.156440285841626)
(1827, 0, 0, 51.239575568834944)
(18